In [1]:
#生成 RSA 密钥对
from OpenSSL import crypto

# 生成密钥对
key = crypto.PKey()
key.generate_key(crypto.TYPE_RSA, 2048)

# 保存私钥
with open("private_key.pem", "wb") as f:
    f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, key))

# 保存公钥
with open("public_key.pem", "wb") as f:
    f.write(crypto.dump_publickey(crypto.FILETYPE_PEM, key))


In [2]:
#创建自签名证书
from OpenSSL import crypto
from datetime import datetime

# 创建证书对象
cert = crypto.X509()

# 设置证书的属性
cert.get_subject().C = "CN" # 国家
cert.get_subject().ST = "Beijing" # 省份
cert.get_subject().L = "Beijing" # 城市
cert.get_subject().O = "My Organization" # 组织
cert.get_subject().OU = "My Unit" # 组织单位
cert.get_subject().CN = "localhost" # 通用名称

# 设置证书的序列号和有效期
cert.set_serial_number(1000) # 序列号
cert.gmtime_adj_notBefore(0) # 有效期起始时间
cert.gmtime_adj_notAfter(365*24*60*60)  # 有效期结束时间 365天

# 设置公钥
cert.set_pubkey(key)

# 自签名
cert.sign(key, "sha256")

# 保存证书
with open("certificate.crt", "wb") as f:
    f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert))


In [ ]:
#加密和解密数据
"""pyOpenSSL 主要用于处理证书和密钥管理，直接进行数据加密和解密的功能有限。对于数据的加解密，我们使用 cryptography 库。"""
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

# 加载公钥
with open("public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read())

# 加密数据
message = b"Hello, OpenSSL!"
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),# 生成函数 mgf
        algorithm=hashes.SHA256(), # 哈希算法
        label=None # 标签
    )
)

# 加载私钥
with open("private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(f.read(), password=None) # 无密码

# 解密数据
plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print(plaintext.decode())

